<span style="float:left;">Licence CC BY-NC-ND</span><span style="float:right;">François Rechenmann &amp; Thierry Parmentelat&nbsp;<img src="media/inria-25.png" style="display:inline"></span><br/>

# Tableau des distances

Dans ce complément nous allons voir comment implémenter l'algorithme de calcul du tableau des distances entre toutes les espèces d'un ensemble donné.

In [ ]:
# la formule magique pour utiliser print() en python2 et python3
from __future__ import print_function
# pour que la division se comporte en python2 comme en python3
from __future__ import division

### Fichier texte

Comme dans la vidéo, nous supposons que nous disposons d'un fichier qui contient les séquences ADN des espèces auxquelles on s'intéresse.

Avant de voir les outils offerts par python pour lire un tel fichier, voici le contenu du fichier qui nous servira pour faire tourner notre algorithme&nbsp;:

In [ ]:
cat data/species.txt

Il est très simple de lire un tel fichier à partir d'un programme python&nbsp;:

In [ ]:
with open('data/species.txt') as input:
    for line in input:
        print(line)

Vous remarquez que les lignes sont séparées par une ligne blanche, qui est liée au fait que la variable `line` contient déjà le caractère *newline* de fin de ligne présent dans le fichier, auquel s'ajoute celui que `print` affiche systématiquement. Pour éviter cela on peut s'y prendre de deux façons.

##### `print` sans *newline*

La première méthode consiste à empêcher `print` de mettre ce caractère *newline*&nbsp;:

In [ ]:
with open('data/species.txt') as input:
    for line in input:
        # on laisse le newline dans line, mais
        # avec end="", print n'affiche pas son newline
        # automatique à la fin de l'appel
        print(line, end="""""")

##### Enlever les *newline* 

L'autre solution consiste à enlever *newline* de la variable `line`, c'est ce que nous allons faire ici pour être compatibles avec les algorithmes écrits jusqu'ici&nbsp;:

In [ ]:
with open('data/species.txt') as input:
    for line in input:
        # on enlève la fin de ligne directement dans line
        line = line.strip()
        # et donc on peut imprimer normalement
        print(line)

##### Les numéros de ligne avec `enumerate`

On peut utiliser également `enumerate`, comme on a déjà eu l'occasion de le faire, afin d'accéder à un compteur de ligne - sauf que, comme toujours en python, celui-ci va commencer à `0`, ce qui en fait nous arrange bien. Ce qui donne&nbsp;:

In [ ]:
with open('data/species.txt') as input:
    for index, line in enumerate(input):
        # on enlève la fin de ligne directement dans line
        line = line.strip()
        # et donc on peut imprimer normalement
        print(index, line)

### Distance de Needleman et Wunsch

Nous importons la fonction `distance` telle que nous l'avions écrite la semaine passée, dans la séquence 9 sur la version itérative de l'algorithme de Needleman et Wunsch&nbsp;:

In [ ]:
from w4_s09_c1_needleman_wunsh_iter import needleman_wunsch, distance

Et pour rappel, à titre d'exemple&nbsp;:

In [ ]:
sample1 = "ACCTCTGTATCTATTCGGCATCGATCAT"
sample2 = "ACCTCGTGTATCTCTTCGGCATCATCAT"

needleman_wunsch(sample1, sample2)

In [ ]:
# et en effet
distance(sample1, sample2)

### Dictionnaire indexé par des tuples (version simplifiée)

Pour ceux qui auraient choisi de sauter la section consacrée à ce sujet dans le complément sur Needleman et Wunsch, voici en version courte ce qu'il suffit de savoir pour comprendre l'algorithme de cette section.

En version courte, on peut créer un dictionnaire&nbsp;:

In [ ]:
# à partir d'un dictionnaire 
d = {}

# on a vu qu'on peut y insérer des clés qui sont des entiers ou des chaines
d[1] = "un"
d["deux"] = 2
print(d)

Eh bien on peut tout aussi bien y insérer des clés qui sont des tuples, et cela se présente comme ceci&nbsp;:

In [ ]:
d [ (1, 2) ] = "le couple 1,2"
print(d)

Il n'y a aucune sorte de restriction, on utilise un dictionnaire exactement comme d'habitude, en utilisant le même tuple pour retrouver le résultat&nbsp;: 

In [ ]:
d [ (1, 2)]

Cette technique nous intéresse ici de façon à économiser de la mémoire&nbsp;; en effet on a vu dans la vidéo que le tableau à calculer est symétrique, on n'a donc pas besoin de créer une matrice complète. Nous verrons dans la prochaine séquence une utilisation encore plus intéressante de ce trait, mais n'anticipons pas.

### Calcul du tableau des distances

Avec tous ces outils à notre disposition, il est très simple d'écrire la fonction qui calcule le tableau de toutes les distances, en voici le code&nbsp;:

In [ ]:
def all_distances(filename):
    """
    Lit le fichier d'entrée, qui doit contenir une séquence ADN par ligne
    
    Retourne en valeur:
    * une liste des séquences d'entrée
    * un dictionnaire indexé sur les couples d'indices, et qui contient la distance associée
      à ce couple d'entrées
    """
    # on commence par lire le fichier et ranger toutes les entrées dans un seul tableau
    
    adns = []
    distances = {}
    
    with open(filename) as input:
        for line in input:
            adns.append(line.strip())
            
    for i, adn1 in enumerate(adns):
        for j in range(i):
            distances[ (i, j)] = distance(adns[i], adns[j])

    return adns, distances

In [ ]:
all_distances("data/species.txt")

Les seuls petits inconvénient avec cette technique sont que&nbsp;:
  * (a) dans le dictionnaire, on perd l'ordre dans lequel sont insérées les valeurs,
  * (b) et aussi bien sûr, on doit créer le tuple 'dans le bon sens' c'est-à-dire avec $i>j$. 

Voici comment on peut par exemple envisager pallier à ces inconvénients&nbsp;:

In [ ]:
def get_distance(d, i, j):
    return 0 if i == j \
        else d[(i, j)] if i > j \
        else d[(j, i)]

# on affiche sur 4 caractères
space = 4*" "
formatr = "{:4}"
formatl = "{:<4}"

def pretty_distances(filename):
    adns, distances = all_distances(filename)
    l = len(adns)
    # première ligne
    print(space + "".join([ formatr.format(i) for i in range(l)]))
    # pour chaque ligne
    for i in range(l):
        print(formatl.format(i) 
              + "".join([formatr.format(get_distance(distances, i, j)) 
                                   for j in range(l)]))

In [ ]:
pretty_distances("data/species.txt")